In [1]:
import spacy
from common_files.recipe_handler import Recipe, parameterize_ingredient_phrase, ComplexEncoder
from common_files.mogodb_functions import postRecipe
from common_files.s3_functions import upload_image_to_s3
import re
from PIL import Image
from io import StringIO
from datetime import datetime, timedelta
import requests
import csv
import json
from bson import json_util
import time
import os
from json_tricks import dump, dumps, load, loads, strip_comments
from elasticsearch import Elasticsearch
nlp = spacy.load('en_core_web_sm')

C:\Users\punee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_csv(filename):
    rows = []
    f = open(filename,encoding='utf-8')
    reader = csv.reader(f)
    for row in reader:
        rows.append(row)
    f.close()
    return rows

In [18]:
data = load_csv(r"C:\Users\punee\Downloads\archive\cuisines.csv")
del data[0]

# data = load_csv(r"C:\Users\punee\Downloads\archive (1)\RecipeNLG_dataset.csv")
# print(data[0])


In [19]:
data = data[1984:]
print(len(data))

2252


In [20]:
es = Elasticsearch(
    "https://recipe-bhandaar.es.us-central1.gcp.cloud.es.io:9243/",
    basic_auth=('elastic', 'mTOFdE47BJOcrdCHZ7RKXv5l'),
)

In [21]:
for i,row in enumerate(data):
    print("ID: ",i)
    #Recipe_title
    recipe_title_raw = row[0]
    print(recipe_title_raw)
    recipe_splits = re.split(r'[`!@#$%^*()_+\-=\[\]{};\':"\\|,.<>\/?~]',recipe_title_raw)
    for split in recipe_splits:
        if ((not split.isascii()) or split == 'Recipe In Hindi'):
            recipe_splits.remove(split)
    while '' in recipe_splits:
        recipe_splits.remove('')
    print(recipe_splits)
    try:
        recipe_title = recipe_splits[0].replace("Recipe","").strip()
    except:
        recipe_title = recipe_title_raw
    try:
        recipe_alt_title = recipe_splits[1].replace("Recipe","").strip()
    except:
        recipe_alt_title=""
    # print("recipe_title: ",recipe_title)
    # print("recipe_alt_title: ",recipe_alt_title)
    recipe_object = Recipe(recipe_title,recipe_alt_title)

    #Recipe_img 
    image_url = row[1]   
    # print(image_url)
    r = requests.get(image_url).content
    image_name = recipe_title.replace(" ","_") + "_" + time.strftime("%Y%m%d-%H%M%S") + ".jpg"
    with open (image_name, 'wb') as f:
        f.write(r)
    upload_image_to_s3(image_name,image_name) #stores image in s3
    print(image_name)
    os.remove(image_name)
    recipe_media_contents = Recipe.MediaContent("img",image_name)
    recipe_object.media_contents = recipe_media_contents

    #Recipe description 
    description_raw = row[2]
    doc = nlp(description_raw)
    description_paragraphs = [sent.text.strip() for sent in doc.sents]
    while '' in description_paragraphs:
        description_paragraphs.remove('')
    # print(description_paragraphs)
    recipe_description = Recipe.Description("",description_paragraphs)

    #Recipe cuisines
    cuisine = row[3]
    cuisine=cuisine.replace("Recipes","").strip()
    # print(cuisine)
    
    #Recipe meal type
    meal_type = row[4]
    # print(meal_type)

    #dietary info
    dietary_info_raw = row[5]
    dietary_infos = dietary_info_raw.split(',')
    
    #meal time
    time_to_cook= row[6].replace("Total in ","").replace(" M","")
    # print(time_to_cook)

    recipe_metas = Recipe.Meta(cuisine,time_to_cook, "0", "", dietary_infos,0.0, meal_type, "")
    
    #ingredient 
    ingredients_raw = row[7]
    ingredient_splits = ingredients_raw.replace('\t','').split('\n')
    ingredients = []
    while '' in ingredient_splits:
        ingredient_splits.remove('')
    # print(ingredient_splits)
    for split in ingredient_splits:
        split_pr = split.strip()
        ingredient_params=parameterize_ingredient_phrase(split_pr)
        # print(ingredient_params)
        if len(ingredient_params[0])>0:
            ingredient_object=Recipe.Ingredient("",ingredient_params[0],ingredient_params[1],ingredient_params[2],ingredient_params[3])
            ingredients.append(ingredient_object)
    recipe_object.ingredients = ingredients

    #steps
    step_raw = row[8]
    doc = nlp(step_raw)
    step_sents = [sent.text.strip() for sent in doc.sents]
    # print(step_sents)
    step_object = Recipe.Step(
                    step_sents, "", timedelta(minutes=10), "", "", "")
    recipe_object.steps(step_object)

    doc = json.loads(dumps(recipe_object, primitives=True, indent=4))
    doc_serialized = json.dumps(doc,default = json_util.default)
    print(es.index(index="search-recipes", id=(234+i), document=doc_serialized))
    
    # print("-"*100)
    # if i==0:
    #     break



ID:  0
चावल बनाने की विधि रेसिपी - Steamed Rice Recipe 
[' Steamed Rice Recipe ']
Steamed_Rice_20230109-002756.jpg
{'_index': 'search-recipes', '_id': '234', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1525, '_primary_term': 2}
ID:  1
Pumpkin Erissery Recipe - Kerala Pumpkin Coconut Curry 
['Pumpkin Erissery Recipe ', ' Kerala Pumpkin Coconut Curry ']
Pumpkin_Erissery_20230109-002759.jpg
{'_index': 'search-recipes', '_id': '235', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1527, '_primary_term': 2}
ID:  2
कल्याना वीतु सेनाई करी रेसिपी - Kalyana Veetu Senai Curry Recipe
[' Kalyana Veetu Senai Curry Recipe']
Kalyana_Veetu_Senai_Curry_20230109-002801.jpg
{'_index': 'search-recipes', '_id': '236', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1528, '_primary_term': 2}
ID:  3
Sukhi Suran Masala Sabzi Recipe
['Sukhi Suran 

KeyboardInterrupt: 

In [20]:
print("No Onion No Garlic (Sattvic)".split(','))

['No Onion No Garlic (Sattvic)']
